In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings 

warnings.filterwarnings("ignore")

In [ ]:
os.chdir("..")
from pipeline.preprocesamiento import obtener_datos_bcrp

In [ ]:
from pipeline.config import SERIE_ORO, RUTA_ORO, SERIE_AFP, SERIE_TC, RUTA_AFP, RUTA_TC
from pipeline.preprocesamiento import limpiar_datos_bcrp
df_oro = obtener_datos_bcrp(SERIE_ORO)
df_afp = obtener_datos_bcrp(SERIE_AFP)
df_tc = obtener_datos_bcrp(SERIE_TC)
limpiar_datos_bcrp(df_oro, False, False, RUTA_ORO) 
limpiar_datos_bcrp(df_afp, True, False, RUTA_AFP)
limpiar_datos_bcrp(df_tc, False, True, RUTA_TC)

In [ ]:
# Analizar datos

df_oro = pd.read_csv(RUTA_ORO)
df_afp = pd.read_csv(RUTA_AFP)
df_tc = pd.read_csv(RUTA_TC)

In [ ]:
# Analizar la tendencia del oro

sns.lineplot(df_oro, x = "periodo_limpio", y= "valor_limpio")
plt.title("Tendencia del oro")
plt.xlabel("Fecha")
plt.ylabel("Valor del oro")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Calcular el porcentaje de retorno anual

df_oro["retorno_anual"] = df_oro["valor_limpio"].pct_change(periods=12) 


sns.lineplot(
    df_oro[df_oro["retorno_anual"].isna() == False],
    x = "periodo_limpio",
    y= "retorno_anual",
)

plt.title("Tendencia del retorno anual del oro")
plt.xlabel("Fecha")
plt.ylabel("Retorno anual")
plt.xticks(rotation=45)
plt.show()

In [ ]:

sns.lineplot(
    df_afp, 
    x = "periodo_limpio", 
    y= "valor_limpio",
    label = "AFP")
sns.lineplot(
    df_oro[df_oro["retorno_anual"].isna() == False],
    x = "periodo_limpio",
    y= "retorno_anual",
    label = "Oro",
)

#plt.title("Tendencia de los fondos de pensiones")
#plt.xlabel("Fecha")
#plt.ylabel("Valor de los fondos de pensiones")
plt.xticks(rotation=45)
plt.show()

In [ ]:
df_spy = pd.read_csv("data/raw/spy500.csv", sep = ";", parse_dates=["periodo"])
## Calcular promedio mensual
# Convertir la columna "periodo" a tipo datetime

df_spy["periodo"] = pd.to_datetime(df_spy["periodo"], format="%d/%m/%Y")

df_spy_final = (
    df_spy
    .groupby(df_spy["periodo"].dt.to_period("M"))
    .mean(numeric_only=True)
    .reset_index(names="periodo_clean")
)
df_spy_final["periodo_clean"] = df_spy_final["periodo_clean"].dt.to_timestamp()
# Convertir "periodo_clean" a primer día del mes
df_spy_final["periodo_clean"] = pd.to_datetime(df_spy_final["periodo_clean"]).values.astype('datetime64[M]')

# Calcular el retorno anual
df_spy_final["retorno_anual"] = df_spy_final["valor SPY"].pct_change(periods=12)
df_spy_final["periodo_limpio"] = df_spy_final["periodo_clean"].astype(str)

In [ ]:
df_afp.rename(columns = {"valor_limpio": "valor_afp"}, inplace = True)
df_oro.rename(columns = {"retorno_anual": "retorno_oro"}, inplace = True)
df_spy_final.rename(columns = {"retorno_anual": "retorno_spy"}, inplace = True)

In [ ]:
df_final = df_spy_final.merge(df_afp, how = "left", on = "periodo_limpio").merge(df_oro, how = "left", on = "periodo_limpio").drop(columns = ["periodo_clean"])

df_final[df_final["periodo_limpio"]> "2023-03-01"]

In [ ]:
sns.lineplot(
    df_final[df_final["periodo_limpio"]> "2023-03-01"], 
    x = "periodo_limpio", 
    y= "retorno_spy",
    label = "spy")
sns.lineplot(
    df_final[df_final["periodo_limpio"]> "2023-03-01"], 
    x = "periodo_limpio", 
    y= "valor_afp",
    label = "afp")

sns.lineplot(
    df_final[df_final["periodo_limpio"]> "2023-03-01"], 
    x = "periodo_limpio", 
    y= "retorno_oro",
    label = "oro")
#plt.title("Tendencia de los fondos de pensiones")
#plt.xlabel("Fecha")
plt.ylabel("Retorno")
plt.xticks(rotation=45)
plt.show()

In [ ]:
df_oro.dtypes